<a href="https://colab.research.google.com/github/AdriCon02/Deep-Fore/blob/main/Deep-Fore/Deep_Fore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep-Fore APP

In [ ]:
!pip install gradio
!pip install supervision
from PIL import ImageDraw
import numpy as np

In [ ]:
def load():
    from google.colab import files
    uploaded = files.upload()
    for filename in uploaded.keys():
        return Image.open(filename)

# Dividir la imagen en 24 mosaicos (4 filas x 4 columnas)
def split(imagen, filas, columnas):
    ancho, alto = imagen.size
    ancho_mosaico = ancho // columnas
    alto_mosaico = alto // filas
    mosaicos = []
    for fila in range(filas):
        for columna in range(columnas):
            izquierda = columna * ancho_mosaico
            superior = fila * alto_mosaico
            derecha = (columna + 1) * ancho_mosaico
            inferior = (fila + 1) * alto_mosaico
            mosaico = imagen.crop((izquierda, superior, derecha, inferior))
            mosaicos.append(mosaico)
    return mosaicos

from PIL import Image, ImageDraw

def join(imagen_original, resultados_mosaicos, tamano_mosaico, model1_labels, model2_labels, label_colors):
    # Crear una nueva imagen del mismo tamaño que la imagen original
    imagen_anotada = Image.new('RGB', imagen_original.size)
    # Copiar la imagen original a la imagen anotada
    imagen_anotada.paste(imagen_original, (0, 0))
    # Inicializar un objeto de dibujo para dibujar las anotaciones
    draw = ImageDraw.Draw(imagen_anotada)

    # Definir el número total de mosaicos
    num_mosaicos = len(resultados_mosaicos)

    # Definir el número de filas y columnas de mosaicos
    num_filas = 4
    num_columnas = 4

    for mosaico_idx in range(num_mosaicos):
        # Calcular las coordenadas del mosaico actual en la imagen original
        fila_idx = mosaico_idx // num_columnas
        columna_idx = mosaico_idx % num_columnas
        x_start = columna_idx * tamano_mosaico[1]  # Ancho del mosaico
        y_start = fila_idx * tamano_mosaico[0]  # Altura del mosaico

        # Obtener los resultados de ambos modelos para el mosaico actual
        resultados_modelos = resultados_mosaicos[mosaico_idx]

        # Dibujar las anotaciones del mosaico actual en la imagen anotada
        for resultados in resultados_modelos:
            for resultado in resultados:
                boxes = resultado.boxes.xyxy
                labels = resultado.boxes.cls
                model_index = resultados_modelos.index(resultados)  # Obtener el índice del modelo actual

                for box, label in zip(boxes, labels):
                    x1, y1, x2, y2 = box.cpu().numpy()  # Obtener las coordenadas de la caja delimitadora
                    area = (x2-x1)*(y2-y1)

                    if area >= 500:
                        # Ajustar las coordenadas del cuadro delimitador al marco del mosaico actual
                        x1 += x_start
                        y1 += y_start
                        x2 += x_start
                        y2 += y_start

                        # Obtener el color correspondiente a la etiqueta
                        if model_index == 0:  # Si es el primer modelo
                            if label.item() == 0:  # Si es label 0
                                color = 'red'
                            else:  # Si es label 1
                                color = 'green'
                        else:  # Si es el segundo modelo
                            if label.item() == 0:  # Si es label 0
                                color = 'yellow'
                            else:  # Si es label 1
                                color = 'blue'

                        # Dibujar el cuadro delimitador en la imagen anotada con el color correspondiente
                        draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=2)

    return imagen_anotada

# def nms(boxes, scores, iou_threshold=0.5):
#     # Implementación de NMS usando PyTorch
#     keep = []
#     if boxes.numel() == 0:
#         return torch.tensor(keep, dtype=torch.long)

#     x1 = boxes[:, 0]
#     y1 = boxes[:, 1]
#     x2 = boxes[:, 2]
#     y2 = boxes[:, 3]

#     areas = (x2 - x1 + 1) * (y2 - y1 + 1)
#     _, order = scores.sort(0, descending=True)

#     while order.numel() > 0:
#         if order.numel() == 1:
#             i = order.item()
#             keep.append(i)
#             break

#         i = order[0].item()
#         keep.append(i)

#         xx1 = x1[order[1:]].clamp(min=x1[i])
#         yy1 = y1[order[1:]].clamp(min=y1[i])
#         xx2 = x2[order[1:]].clamp(max=x2[i])
#         yy2 = y2[order[1:]].clamp(max=y2[i])

#         w = (xx2 - xx1 + 1).clamp(min=0)
#         h = (yy2 - yy1 + 1).clamp(min=0)
#         inter = w * h

#         ovr = inter / (areas[i] + areas[order[1:]] - inter)

#         inds = (ovr <= iou_threshold).nonzero(as_tuple=False).squeeze()
#         order = order[inds + 1]

#     return torch.tensor(keep, dtype=torch.long)

#     # Función para dibujar las cajas delimitadoras
# def draw_boxes(results, label_map, color, confidence_threshold, iou_threshold=0.5, label_offset=0):
#     for result in results:
#         boxes = result.boxes.xyxy
#         scores = result.boxes.conf
#         labels = result.boxes.cls

#         # Filtrar por umbral de confianza
#         valid = scores >= confidence_threshold
#         boxes = boxes[valid]
#         scores = scores[valid]
#         labels = labels[valid]

#         # Aplicar NMS
#         keep = nms(boxes, scores, iou_threshold)
#         boxes = boxes[keep]
#         scores = scores[keep]
#         labels = labels[keep]

#         for box, label, score in zip(boxes, labels, scores):
#             x1, y1, x2, y2 = box  # Obtener las coordenadas de la caja delimitadora
#             rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor=color, facecolor='none')
#             ax.add_patch(rect)
#             ax.text(x1, y1 + label_offset, f"{label_map[int(label)]}", color=color, fontsize=12, verticalalignment='top')  # Agregar el label con la letra correspondiente

# Función para validar la entrada de la imagen

# Función para validar el ancho y el alto
def validate_resolution(width, height):
    if width != 1360 or height != 1024:
        return False
    return True

def NHG_parameters(results1, results2, NHG_params):
    # Inicializar la variable para almacenar el área total de las cajas con label 3
        num_tumor_labels = NHG_params[0]
        area_tumor_total = NHG_params[1]
        num_mitosis_labels = NHG_params[2]
        area_lumen_total = NHG_params[3]
        area_tumor_total_x = NHG_params[4] #Es una area especial que no tiene en cuenta el filtrado para la media, cuenta las areas pequeñas para el lumen tb :)

        # Procesar los resultados del primer modelo
        for result in results1:
            boxes = result.boxes.xyxy
            labels = result.boxes.cls
            for box, label in zip(boxes, labels):
                if int(label) == 1:  # Verificar si el label es 1 (Tumoral nuclei)
                    x1, y1, x2, y2 = box
                    area = (x2 - x1) * (y2 - y1)
                    area_tumor_total_x += area
                    if area >= 750:
                        num_tumor_labels += 1
                        area_tumor_total += area

                elif int(label) == 0:  # Verificar si el label es 0 (Mitosis)
                    num_mitosis_labels += 1

        # Procesar los resultados del segundo modelo
        for result in results2:
            boxes = result.boxes.xyxy
            labels = result.boxes.cls
            for box, label in zip(boxes, labels):
                if int(label) == 0:  # Verificar si el label es 0 (Lumen)
                    x1, y1, x2, y2 = box
                    area = (x2 - x1) * (y2 - y1)
                    area_lumen_total += area

        NHG_params=[num_tumor_labels,area_tumor_total,num_mitosis_labels,area_lumen_total,area_tumor_total_x]

        return NHG_params

def mc_score(mitotic_count):

  if mitotic_count >= 20:
    mitosis_score = 3
  elif 10<=mitotic_count<19:
    mitosis_score = 2
  else:
    mitosis_score = 1

  return mitosis_score

def np_score(avg_tumor_area,microscope_resolution,checkboxes):

  if checkboxes == "Post-menopausia":
      epitelial_cell_area = 48 #In microns**2
  elif checkboxes == "Madurez sexual":
      epitelial_cell_area = 75 #In microns**2
  elif checkboxes == "Embarazada":
      epitelial_cell_area = 118 #In microns**2
  else:
      epitelial_cell_area = 79 #In microns**2

  prop_height = 256/512
  prop_width = 360/640

  height_resolution = microscope_resolution[0]*prop_height
  width_resolution = microscope_resolution[1]*prop_width

  area_per_pixel = height_resolution*width_resolution

  epitelial_cell_pixels = epitelial_cell_area/area_per_pixel

  size_ratio = avg_tumor_area/epitelial_cell_pixels

  print(epitelial_cell_pixels)

  if size_ratio >= 1.5:
    pleomorphism_score = 3
  elif 1.1<=size_ratio<1.5:
    pleomorphism_score = 2
  else:
   pleomorphism_score = 1

  return pleomorphism_score

def tf_score(pctg_lumen):

  if pctg_lumen <= 10:
    lumen_score = 3
  elif 10<=pctg_lumen<75:
    lumen_score = 2
  else:
    lumen_score = 1

  return lumen_score

def NHG_score(mitosis_score,pleomorphism_score,lumen_score):

  total_score=mitosis_score+pleomorphism_score+lumen_score

  if total_score > 7:
    NHG_result = 'Grado III'
  elif 5<total_score<=7:
    NHG_result = 'Grado II'
  else:
    NHG_result = 'Grado I'

  print(f"El tumor analizado es de {NHG_result}, scoring: MC-> {mitosis_score}, NP-> {pleomorphism_score},TF-> {lumen_score};TOTAL -> {total_score}")

  return NHG_result

In [ ]:
!nvidia-smi

import os
HOME = os.getcwd()
print(HOME)

!pip install -U ultralytics==8.0.196

from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import supervision as sv


model1 = YOLO('/content/best_tm_mc.pt')
model2 = YOLO('/content/best_lum.pt')

In [ ]:
import gradio as gr
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image

# Función predict de ejemplo (debes reemplazar esto con tu propia función predict)


# Definir la función que manejará la entrada y salida de datos
def predict(imagen, num1, num2, checkboxes):

    if not validate_resolution(imagen.size[0], imagen.size[1]):
        gr.Info("ERROR: Compruebe que las dimensiones de la imagen son 1360x1024.")
        return None

    gr.Info("¡Predicción en proceso! Espere unos segundos...")
    mosaicos = split(imagen, 4, 4)
    area_img_total = imagen.size[0] * imagen.size[1]
    area_tumor_total = 0
    area_lumen_total = 0

    num_tumor_labels = 0
    num_mitosis_labels = 0

    NHG_params=[num_tumor_labels,area_tumor_total,num_mitosis_labels,area_lumen_total]

    # Resultados de la detección de objetos
    resultados_mosaicos = []

    # Diccionario de colores para cada etiqueta
    # Diccionarios de mapeo para los labels de cada modelo
    model1_labels = {0: 'M', 1: 'T'}
    model2_labels = {0: 'L', 1: 'NL'}
    label_colors = {'M': 'blue', 'T': 'red', 'L': 'green', 'NL': 'yellow'}

    # Iterar por todos los mosaicos
    for idx, mosaico in enumerate(mosaicos):
        # Realizar la detección de objetos en el mosaico seleccionado con ambos modelos
        results1 = model1(mosaico, conf = 0.1, iou = 0.2)
        results2 = model2(mosaico, conf = 0.7, iou = 0.6)

        # Agregar los resultados a la lista
        resultados_mosaicos.append((results1, results2))

        # # Mostrar el mosaico
        # plt.imshow(mosaico)
        # ax = plt.gca()

        # # Dibujar las cajas delimitadoras para ambos modelos
        # draw_boxes(results1, model1_labels, 'r', 0.207, 0.5)  # Dibujar las cajas del primer modelo en rojo
        # draw_boxes(results2, model2_labels, 'b', 0.244, 0.5, label_offset=15)  # Dibujar las cajas del segundo modelo en azul, con un desplazamiento de la etiqueta

        # # Ocultar los ejes
        # plt.axis('off')
        # plt.title(f"Mosaico {idx + 1}")  # Agregar un título con el número de mosaico
        # plt.show()

        NHG_params= NHG_parameters(results1, results2, NHG_params)

        avg_tumor_area = NHG_params[1].item() / NHG_params[0] if NHG_params[0] > 0 else 0

    # # Unir los mosaicos y agregar las anotaciones a la imagen original
    imagen_anotada = join(imagen, resultados_mosaicos, [256,340],model1_labels, model2_labels,label_colors) #CAMBIAR Y PARAMETRIZAR

    pctg_lumen = (NHG_params[3] / NHG_params[1]) * 100
    mitosis_count = NHG_params[2]

    mitosis_score = mc_score(mitosis_count)
    pleomorphism_score = np_score(avg_tumor_area,[num1,num2],checkboxes)
    lumen_score = tf_score(pctg_lumen)

    grado = NHG_score(mitosis_score,pleomorphism_score,lumen_score)

    return imagen_anotada, mitosis_score, pleomorphism_score, lumen_score,grado,avg_tumor_area


# Cargar la imagen en el entorno de Colab
# image_path = "/content/deepfore.PNG"
# image = Image.open(image_path)

# Crear la interfaz de usuario con Gradio
with gr.Blocks() as iface:
    # with gr.Row():
    #     gr.Image(image, height = 400, width = 800, show_label = False, show_download_button = False)

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="PNG/JPG con aumento 40x")
            input_width = gr.Number(label="Resolución microscópica: Ancho (µm per px)")
            input_height = gr.Number(label="Resolución microscópica: Alto (µm per px)")
            input_stage = gr.Radio(["Post-menopausia", "Madurez sexual", "Embarazada", "Consumidora de inhibidores de ovulación"], label="Seleccione una etapa:")
            submit_button = gr.Button("Enviar")

        with gr.Column():
            output_image = gr.Image(type="pil", label="Imagen Procesada")
            output_mc = gr.Textbox(label="Actividad mitótica (1-3)")
            output_np = gr.Textbox(label="Pleomorfismo nuclear (1-3)")
            output_tf = gr.Textbox(label="Formación tubular (1-3)")
            output_score = gr.Textbox(label="Grado")
            test = gr.Textbox(label="T-AREA")

    # Conectar la función predict a los componentes de entrada y salida
    submit_button.click(predict, inputs=[input_image, input_width, input_height, input_stage], outputs=[output_image, output_mc, output_np, output_tf, output_score,test])

# Mostrar la interfaz
iface.launch()



In [ ]:
# Definir la función que manejará la entrada y salida de datos
def predict_all(imagen, NHG_params):

    mosaicos = split(imagen, 4, 4)
    area_img_total = imagen.size[0] * imagen.size[1]


    # Resultados de la detección de objetos
    resultados_mosaicos = []

    # Diccionario de colores para cada etiqueta
    # Diccionarios de mapeo para los labels de cada modelo
    model1_labels = {0: 'M', 1: 'T'}
    model2_labels = {0: 'L', 1: 'NL'}
    label_colors = {'M': 'blue', 'T': 'red', 'L': 'green', 'NL': 'yellow'}

    # Iterar por todos los mosaicos
    for idx, mosaico in enumerate(mosaicos):
        # Realizar la detección de objetos en el mosaico seleccionado con ambos modelos
        results1 = model1(mosaico, conf = 0.342, iou = 0.5)
        results2 = model2(mosaico, conf = 0.348, iou = 0.5)

        # Agregar los resultados a la lista
        resultados_mosaicos.append((results1, results2))

        # # Mostrar el mosaico
        # plt.imshow(mosaico)
        # ax = plt.gca()

        # # Dibujar las cajas delimitadoras para ambos modelos
        # draw_boxes(results1, model1_labels, 'r', 0.207, 0.5)  # Dibujar las cajas del primer modelo en rojo
        # draw_boxes(results2, model2_labels, 'b', 0.244, 0.5, label_offset=15)  # Dibujar las cajas del segundo modelo en azul, con un desplazamiento de la etiqueta

        # # Ocultar los ejes
        # plt.axis('off')
        # plt.title(f"Mosaico {idx + 1}")  # Agregar un título con el número de mosaico
        # plt.show()

        NHG_params= NHG_parameters(results1, results2, NHG_params)

    return NHG_params


def obtain_grade(num1, num2, state, NHG_params):

    avg_tumor_area = NHG_params[1].item() / NHG_params[0] if NHG_params[0] > 0 else 0

    pctg_lumen = (NHG_params[3] / NHG_params[4]) * 100
    print(pctg_lumen)
    mitosis_count = NHG_params[2]

    mitosis_score = mc_score(mitosis_count)
    pleomorphism_score = np_score(avg_tumor_area,[num1,num2],state)
    lumen_score = tf_score(pctg_lumen)

    grado = NHG_score(mitosis_score,pleomorphism_score,lumen_score)

    return mitosis_score, pleomorphism_score, lumen_score,grado





In [ ]:
import os
from PIL import Image

# Ejemplo de uso
folder_path = '/content/1'
NHG_params = [0, 0, 0, 0, 0]  # Valores iniciales para NHG_params
num1 = 0.514
num2 = 0.527
state = "Madurez sexual"

# Lista de imágenes en la carpeta
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]


for image_file in image_files:
        # Cargar la imagen
        image_path = os.path.join(folder_path, image_file)
        imagen = Image.open(image_path)

        # Realizar predicción y obtener NHG_params
        NHG_params = predict_all(imagen, NHG_params)

#NHG_params[3]=0.8*NHG_params[3] adjust due to extra area in bboxes


mitosis_score, pleomorphism_score, lumen_score, grado = obtain_grade(num1, num2, state, NHG_params)



In [ ]:
import os
HOME = os.getcwd()
print(HOME)


!pip install roboflow --quiet

#AUGMENTED MULTICLASS-TILING
#from roboflow import Roboflow
#rf = Roboflow(api_key="UYkhq9VF7hUFyzkp3jLN")
#project = rf.workspace("yolo-brecahad-nbqyh").project("brecahad-yolov8")
#version = project.version(1)
#dataset = version.download("yolov8")

#AUGMENTED SINGLECLASS-TILING
#from roboflow import Roboflow
#rf = Roboflow(api_key="UYkhq9VF7hUFyzkp3jLN")
#project = rf.workspace("yolo-brecahad-nbqyh").project("brecahad-singleclass")
#version = project.version(1)
#dataset = version.download("yolov8")

#AUGMENTED SINGLECLASS
#from roboflow import Roboflow
#rf = Roboflow(api_key="UYkhq9VF7hUFyzkp3jLN")
#project = rf.workspace("yolo-brecahad-nbqyh").project("brecahad-singleclass")
#version = project.version(3)
#dataset = version.download("yolov8")

#AUGMENTED MULTICLASS - with histogram normalization
# from roboflow import Roboflow
# rf = Roboflow(api_key="UYkhq9VF7hUFyzkp3jLN")
# project = rf.workspace("yolo-brecahad-nbqyh").project("brecahad-yolov8")
# version = project.version(3)
# dataset = version.download("yolov8")

#AUGMENTED MULTICLASS - tiling and filtering nulls
#from roboflow import Roboflow
#rf = Roboflow(api_key="UYkhq9VF7hUFyzkp3jLN")
#project = rf.workspace("yolo-brecahad-nbqyh").project("brecahad-yolov8")
#version = project.version(5)
#dataset = version.download("yolov8")

#AUGMENTED MULTICLASS - Tiling 4x4
# from roboflow import Roboflow
# rf = Roboflow(api_key="LcZgU4vmjLbLqJ5IY1Sg")
# project = rf.workspace("brecahad-4icqs").project("brecahad-yolov8-2lvts")
# version = project.version(3)
# dataset = version.download("yolov8")

#SEGMENTATION 4 CLASSES

# from roboflow import Roboflow
# rf = Roboflow(api_key="LcZgU4vmjLbLqJ5IY1Sg")
# project = rf.workspace("brecahad-4icqs").project("yolo4class-o8snp")
# version = project.version(1)
# dataset = version.download("yolov8")

#SEGMENTATION 2 CLASSES (TUMOR & MITOSIS)
# from roboflow import Roboflow
# rf = Roboflow(api_key="UeLjBGp1wL0ROOeH62zO")
# project = rf.workspace("yolo-brecahad-5ejws").project("yolo4class")
# version = project.version(5)
# dataset = version.download("yolov8")


#SEGMENTATION 2 CLASSES (LUMEN & NO LUMEN)
# from roboflow import Roboflow
# rf = Roboflow(api_key="UeLjBGp1wL0ROOeH62zO")
# project = rf.workspace("yolo-brecahad-5ejws").project("yolo4class")
# version = project.version(4)
# dataset = version.download("yolov8")



#SEGMENTATION 1 CLASSES (LUMEN)
from roboflow import Roboflow
rf = Roboflow(api_key="Q5QExDKAB5ZyzzVvfGxj")
project = rf.workspace("yolo-brecahad-har64").project("brecahad-od-1umky")
version = project.version(1)
dataset = version.download("yolov8")



# Customize validation settings
!yolo task=detect mode=val model='/content/best_lum.pt' data='/content/datasets/datasets/datasets/BreCaHad-OD-1/data.yaml' iou=0.5 conf=0.01